# Job | Glue Lab

## Modelo de Produção | AWS Glue

- Modelo de job adaptado aos componentes da AWS.

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_INPUT_PATH', 'S3_TARGET_PATH'])

# Criação dos objetos
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Início do Job
job.init(args['JOB_NAME'], args)


source_file = args['S3_INPUT_PATH']
target_path = args['S3_TARGET_PATH']

# Criação do dataframe
df = glueContext.create_dynamic_frame.from_options(
    
    "s3",
    {
        "paths": [source_file]
    },
    "csv",
    {
        "withHeader": True, "separator": ","
    }
    
    )

# Transformações
only_1934 = df.filter(lambda row: row['anoLancamento']=='1934')

# Outputs

glueContext.write_dynamic_frame.from_options(
    
    frame = only_1934,
    connection_type = "s3",
    connection_options = { "path":target_path },
    format = "parquet"

    )

# Fim do Job
job.commit()

## Transformações 

In [1]:
## Declarações

schema = "nome STRING, sexo STRING, total INT, ano INT"
nomes = spark.read.csv("/home/glue_user/workspace/jupyter_workspace/dados_brutos/nomes.csv", header=True, schema=schema)


## Transformações 
# 1. Colocar os valores da coluna 'nome' em Maiúsculo
from pyspark.sql.functions import upper
nome_maiusc_col = nomes.withColumn("nome", upper(nomes["nome"]))

# 2. Contagem das linhas do DataFrame
linhas = nomes.count()

# 3. Contagem de nomes agrupado por ano e sexo, e ordenado por ano de modo decrescente
from pyspark.sql.functions import count
contagem_nomes = nomes.groupBy("ano", "sexo").agg(count("nome").alias("contagem"))
contagem_nomes_ano = contagem_nomes.orderBy("ano", ascending=False)

# 4. Nome feminino mais registrado e o respectivo ano de registro
from pyspark.sql.functions import *
from pyspark.sql.window import Window

janela_analise = Window.partitionBy("ano").orderBy(desc("total"))

# Criação do DataFrame de interesse
nomes_fem = nomes.filter(col('sexo') == "F")
nfs_mais_registrados = nomes_fem.withColumn("rank", dense_rank().over(janela_analise))
nf_mais_registrado_ano = nfs_mais_registrados.filter(nfs_mais_registrados.rank==1)
nf_mais_registrado_final = nf_mais_registrado_ano.select("nome", "ano", "total").orderBy("total", ascending=False)

# 5. Nome masculino mais registrado e o respectivo ano de registro

# Criação do DataFrame de interesse
nomes_masc = nomes.filter(nomes.sexo == "M")
nms_mais_registrados = nomes_masc.withColumn("rank", dense_rank().over(janela_analise))
nm_mais_registrado_ano = nms_mais_registrados.filter(nms_mais_registrados.rank==1)
nm_mais_registrado_final = nm_mais_registrado_ano.select("nome", "ano", "total").orderBy("total", ascending=False)

# 6. Total de registros para cada ano (Apresentar as 10 primeiras linhas ordenada por Ano de modo crescente)

# Criação do DataFrame de interesse
registros_ano = nomes.groupBy("ano").agg(sum("total").alias("total de registros por ano"))
registros_ordenados = registros_ano.orderBy("ano").limit(10)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Script do Job para o S3

- Este script deverá ser executado na plataforma da AWS Glue.

In [ ]:
import sys
import boto3
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'S3_INPUT_PATH', 'S3_TARGET_PATH'])

# Criação dos objetos
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Início do Job
job.init(args['JOB_NAME'], args)

# 1. Leitura do arquivo no s3
source_file = args['S3_INPUT_PATH']
target_path = args['S3_TARGET_PATH']

# Criação do Dynamic Frame
nomes = glueContext.create_dynamic_frame.from_options(
    
    "s3",
    {
        "paths": [source_file]
    },
    "csv",
    {
        "withHeader": True, "separator": ","
    }
    
)

# 2. Impressão do schema do DataFrame gerado
nomes.schema()

# 3. Colocar os valores da coluna 'nome' em Maiúsculo
maiuscula = lambda nome: nome.upper() if nome is not None else None
nome_maiusc_col = Map.apply(frame=nomes, f = maiuscula)

# Outputs

glueContext.write_dynamic_frame.from_options(
    
    frame = nome_maiusc_col,
    connection_type = "s3",
    connection_options = { "path":target_path },
    format = "json",
    partitionKeys = ["sexo", "ano"]

    )

# Fim do Job
job.commit()

